In [1]:
import os
import findspark
spark_home = os.getenv("SPARK_HOME")
print(spark_home)
findspark.init(spark_home)

/usr/local/spark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('persistencia').getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb49a7a1-18ab-4c2d-9b2b-769f6b3c2dfb;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 378ms :: artifacts dl 9ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.

In [4]:
ruta = "./data"


In [5]:
sc = spark.sparkContext

rdd = sc.parallelize([1,2])
rdd.collect()


[1, 2]

In [6]:
df = spark.read.csv("data/sales_info.csv")

In [7]:
df = spark.read.format("csv"). \
    option("header", "true"). \
    option("inferSchema", "true"). \
    load("data/sales_info.csv")

In [8]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [9]:
from pyspark.sql import Row

nuevaFila = [
    Row("APPL","La empresa de la manzanita"),
    Row("GOOG","Líderes en cloud"),
    Row("FB","Especialistas en redes sociales"),
    Row("MSFT","Gran empresa")
]

In [10]:
spark.sparkContext.parallelize(nuevaFila).toDF().show(truncate=False)

+----+-------------------------------+
|_1  |_2                             |
+----+-------------------------------+
|APPL|La empresa de la manzanita     |
|GOOG|Líderes en cloud               |
|FB  |Especialistas en redes sociales|
|MSFT|Gran empresa                   |
+----+-------------------------------+



In [11]:
dfDescRaw = spark.sparkContext.parallelize(nuevaFila).toDF()

In [12]:
dfDesc = dfDescRaw.select(dfDescRaw[0].alias('Company2'),dfDescRaw[1].alias('Description'))

In [13]:
dfJoined = df.join(dfDesc,(df['Company'] == dfDesc['Company2']),"inner").drop(dfDesc['Company2'])

In [14]:
dfJoined.show(truncate=False)

+-------+-------+-----+-------------------------------+
|Company|Person |Sales|Description                    |
+-------+-------+-----+-------------------------------+
|APPL   | Chris |350.0|La empresa de la manzanita     |
|APPL   |Mike   |750.0|La empresa de la manzanita     |
|APPL   |Linda  |130.0|La empresa de la manzanita     |
|APPL   |John   |250.0|La empresa de la manzanita     |
|GOOG   |Frank  |340.0|Líderes en cloud               |
|GOOG   |Charlie|120.0|Líderes en cloud               |
|GOOG   |Sam    |200.0|Líderes en cloud               |
|FB     |Sarah  |350.0|Especialistas en redes sociales|
|FB     |Carl   |870.0|Especialistas en redes sociales|
|MSFT   |Vanessa|243.0|Gran empresa                   |
|MSFT   |Amy    |124.0|Gran empresa                   |
|MSFT   |Tina   |600.0|Gran empresa                   |
+-------+-------+-----+-------------------------------+



## Persistencia

In [15]:
print(ruta+"/sales_result")

./data/sales_result


In [16]:
!rm -rf ./data/sales_result

In [17]:
dfJoined.write.format("csv") \
    .save(ruta + "/sales_result")

In [18]:
dfJoined.write.format("csv") \
    .mode("overwrite") \
    .save(ruta + "/sales_result")

In [19]:
ls data/sales_result

part-00000-aeb8f15c-701b-4ac7-a1ab-1524d6fc3518-c000.csv
part-00002-aeb8f15c-701b-4ac7-a1ab-1524d6fc3518-c000.csv
part-00005-aeb8f15c-701b-4ac7-a1ab-1524d6fc3518-c000.csv
part-00008-aeb8f15c-701b-4ac7-a1ab-1524d6fc3518-c000.csv
part-00011-aeb8f15c-701b-4ac7-a1ab-1524d6fc3518-c000.csv
_SUCCESS


### Leer ahora la copia guardada

In [20]:
dfSalesDesc = spark.read.format("csv"). \
    option("header", "true"). \
    option("inferSchema", "true"). \
    load(ruta + "/sales_result")

In [21]:
dfSalesDesc.show()

+----+-------+-----+--------------------------+
|APPL|  Chris|350.0|La empresa de la manzanita|
+----+-------+-----+--------------------------+
|APPL|   Mike|750.0|      La empresa de la ...|
|APPL|  Linda|130.0|      La empresa de la ...|
|APPL|   John|250.0|      La empresa de la ...|
|GOOG|Charlie|120.0|          Líderes en cloud|
|GOOG|    Sam|200.0|          Líderes en cloud|
|  FB|   Carl|870.0|      Especialistas en ...|
|MSFT|    Amy|124.0|              Gran empresa|
|MSFT|   Tina|600.0|              Gran empresa|
+----+-------+-----+--------------------------+



In [22]:
dfSalesDesc.count()

8

In [23]:
# Modo append
dfJoined.write.format("csv") \
    .option("header","True") \
    .mode("append") \
    .save(ruta + "/sales_result")

In [24]:
dfSalesDesc = spark.read.format("csv"). \
    option("header", "true"). \
    option("inferSchema", "true"). \
    load(ruta + "/sales_result")

In [25]:
dfSalesDesc.count()

20

## Formato Parquet

In [26]:
dfJoined.show()

+-------+-------+-----+--------------------+
|Company| Person|Sales|         Description|
+-------+-------+-----+--------------------+
|   APPL|  Chris|350.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|   GOOG|  Frank|340.0|    Líderes en cloud|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|     FB|  Sarah|350.0|Especialistas en ...|
|     FB|   Carl|870.0|Especialistas en ...|
|   MSFT|Vanessa|243.0|        Gran empresa|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT|   Tina|600.0|        Gran empresa|
+-------+-------+-----+--------------------+



In [27]:
dfJoined.write.format('parquet').mode('overwrite') \
    .save(ruta + "/sales_result_parquet")

In [28]:
dfFromParquet = spark.read.format('parquet') \
    .load(ruta + "/sales_result_parquet")

In [29]:
dfFromParquet.show()

+-------+-------+-----+--------------------+
|Company| Person|Sales|         Description|
+-------+-------+-----+--------------------+
|   APPL|  Chris|350.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|     FB|  Sarah|350.0|Especialistas en ...|
|     FB|   Carl|870.0|Especialistas en ...|
|   GOOG|  Frank|340.0|    Líderes en cloud|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|   MSFT|Vanessa|243.0|        Gran empresa|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT|   Tina|600.0|        Gran empresa|
+-------+-------+-----+--------------------+



In [30]:
dfJoined.write.format('parquet').mode('overwrite') \
    .option("compression", "uncompressed") \
    .save(ruta + "/sales_result_parquet")

In [31]:
dfFromParquet = spark.read.format('parquet') \
    .load(ruta + "/sales_result_parquet")

In [32]:
dfFromParquet.show()

+-------+-------+-----+--------------------+
|Company| Person|Sales|         Description|
+-------+-------+-----+--------------------+
|   APPL|  Chris|350.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|     FB|  Sarah|350.0|Especialistas en ...|
|     FB|   Carl|870.0|Especialistas en ...|
|   GOOG|  Frank|340.0|    Líderes en cloud|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|   MSFT|Vanessa|243.0|        Gran empresa|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT|   Tina|600.0|        Gran empresa|
+-------+-------+-----+--------------------+



## Escritura de datos en paralelo

In [33]:
dfSalesDesc.show()

+-------+-------+-----+--------------------+
|Company| Person|Sales|         Description|
+-------+-------+-----+--------------------+
|   APPL|  Chris|350.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|   GOOG|  Frank|340.0|    Líderes en cloud|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|     FB|  Sarah|350.0|Especialistas en ...|
|     FB|   Carl|870.0|Especialistas en ...|
|   MSFT|Vanessa|243.0|        Gran empresa|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT|   Tina|600.0|        Gran empresa|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|     FB|   Carl|870.0|Especialistas en ...|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT| 

### Repartition. 

    El número de archivos de datos escritos es dependiente del número de particiones que el DF tiene en el momento de ejecutar la escritura. Por defecto, se escribe un fichero por partición. Para comprobar esto, podemos probar a alterar el número de particiones con el método repartition()

In [34]:
dfJoined.repartition(3) \
    .write.format("csv") \
    .option("header","True") \
    .mode("overwrite") \
    .save(ruta + "/sales_result_part")

In [35]:
ls data/sales_result_part

part-00000-8d85437f-55f3-4f68-97dc-b5b9365b7afd-c000.csv
part-00001-8d85437f-55f3-4f68-97dc-b5b9365b7afd-c000.csv
part-00002-8d85437f-55f3-4f68-97dc-b5b9365b7afd-c000.csv
_SUCCESS


## Partitioning

    Permite crear particiones de los datos para decidir como se almacenan los datos. El uso de particionamiento optimiza la lectura/escritura de datos: cuando queremos leer datos solo para una partición (filtro por partición), tamaño de archivos, ...

In [36]:
dfJoined.show()

+-------+-------+-----+--------------------+
|Company| Person|Sales|         Description|
+-------+-------+-----+--------------------+
|   APPL|  Chris|350.0|La empresa de la ...|
|   APPL|   Mike|750.0|La empresa de la ...|
|   APPL|  Linda|130.0|La empresa de la ...|
|   APPL|   John|250.0|La empresa de la ...|
|   GOOG|  Frank|340.0|    Líderes en cloud|
|   GOOG|Charlie|120.0|    Líderes en cloud|
|   GOOG|    Sam|200.0|    Líderes en cloud|
|     FB|  Sarah|350.0|Especialistas en ...|
|     FB|   Carl|870.0|Especialistas en ...|
|   MSFT|Vanessa|243.0|        Gran empresa|
|   MSFT|    Amy|124.0|        Gran empresa|
|   MSFT|   Tina|600.0|        Gran empresa|
+-------+-------+-----+--------------------+



In [37]:
dfJoined.write.format("csv") \
    .option("header","True") \
    .mode("overwrite") \
    .partitionBy("Company") \
    .save(ruta + "/sales_part_company")

In [38]:
ls -R data/sales_part_company/

data/sales_part_company/:
'Company=APPL'/  'Company=FB'/  'Company=GOOG'/  'Company=MSFT'/   _SUCCESS

'data/sales_part_company/Company=APPL':
part-00002-991d82b3-c77f-4a66-8208-c9ac5d522a5d.c000.csv

'data/sales_part_company/Company=FB':
part-00008-991d82b3-c77f-4a66-8208-c9ac5d522a5d.c000.csv

'data/sales_part_company/Company=GOOG':
part-00005-991d82b3-c77f-4a66-8208-c9ac5d522a5d.c000.csv

'data/sales_part_company/Company=MSFT':
part-00011-991d82b3-c77f-4a66-8208-c9ac5d522a5d.c000.csv


In [39]:
spark.stop()